In [1]:
import numpy as np
import random
import timeit
import math
from scipy.stats import norm
from scipy.stats import sem
from scipy import stats
DEBUG = 0

### Criação da lista de variáveis aleatórias

In [2]:
# Gerará uma lista de variáveis aleatórias a partir da função RANDOM
def G5(n):
    random.seed(0)

    vet_rn = []
    for i in range(0,n,1):
        vet_rn.append(random.random())
    
    return(vet_rn)

In [3]:
# Transforma uma lista de variáveis aleatórias em uma distribuição aleatória uniforme entre [0,1]
def random_list(vet,B):
    Ts = np.zeros(len(vet))
    for i in range(0,len(vet)):
        Ts[i] = -B*np.log(1-vet[i])
    return list(Ts)

### Função para formação do intervalo de entrada

In [4]:
# Soma-se o próximo com o anterior para ter uma noção de tempo de chagada. Sempre o tempo de chegada do
#primeiro será 0.
def sum_intervals(vet):
    for i in range(1, len(vet)):
        vet[i] += vet[i-1]
    return vet

def add_interval(last_vet, vet):
    last = last_vet[len(last_vet)-1]
    for i in range(0, len(vet)):
        last += vet[i]
        last_vet.append(last)
        
    return last_vet

In [5]:
#Normaliza as listas que tiveram o transiente removido
def GenarateNValues(H, U, N, tx_service, tx_entry):
    valuestoGenerate = N - len(tx_service)
    random_arr = G5(valuestoGenerate)
    tx_service = tx_service + random_list(random_arr,U)
    
    bff = random_list(random_arr,H) 
    tx_entry = add_interval(tx_entry, bff)
    
    return tx_service, tx_entry

### Função para calcular intervalo de confiança

In [6]:
# Dado uma lista de valores de processamento MM1, calcula-se com a função stats.t.interval o intervalo
#de confiança para 95%
def calculate_confidence_95(mm1):
    return (stats.t.interval(confidence=0.95, df = len(mm1)-1, loc=np.mean(mm1), scale=stats.sem(mm1)))

def verify_interval(wait_list):
    Table_value=1085#1074
    confidence = calculate_confidence_95(wait_list)
    interval_ok = False
    
    piso = confidence[0]
    teto = confidence[1]
    
    if(DEBUG == 1): print(piso, " ", teto)
    if(Table_value >= piso and Table_value <= teto):
        interval_ok = True
    
    return interval_ok

### MSER-5Y

In [7]:
def ZMean(ZVet, k, d):
    mean = 0
    for i in range(d, k):
        mean += ZVet[i]
        
    mean = mean/(k-d)
    return mean
    
def ZDP(ZVet, k, d, ZVetMean):
    dp = 0
    for i in range(d, k):
        dp += (ZVet[i] - ZVetMean)**2

    dp = dp/(k-d)
    
    return dp
    
def MSER5(k, d, ZVetVar):
    return ZVetVar/(k-d)**(1/2)

def MSER5Y(vet, d):
    N = len(vet)
    m = 5
    k = N/m
    if(DEBUG == 1): 
        print("N = ", len(vet))
        print("m = 5")
        print("k = ", k, "\n")
    
    k = int(k)
    ZVet = np.zeros(k)
    #if(DEBUG == 1): print("Blocos de cálculo do Z")
    for j in range(1,k+1):
        for i in range(1,m+1):
            index = (m*(j-1)+i)-1
            #if(DEBUG == 1):print("index: ",m,"* (",j-1,") +",i, "=", index)
            ZVet[j-1] += vet[index]
        
        #if(DEBUG == 1): print("\n")

    ZVet = ZVet/m
    #if(DEBUG == 1): print("Vetor Z: ", ZVet)
    
    ZVetMean = ZMean(ZVet, k, d)
    if(DEBUG == 1): print("Z Média = ",ZVetMean)
    
    ZVetDP = ZDP(ZVet, k, d, ZVetMean)
    ZVetVar = ZVetDP**(1/2)
    if(DEBUG == 1): print("Z Desvio Padrão = ",ZVetDP)
    if(DEBUG == 1): print("Z Variância = ",ZVetVar)
    
    mser5 = MSER5(k, d, ZVetVar)
    if(DEBUG == 1): print("\nMSER5 = ",mser5)
    
    return mser5

def MSER5YLoop(vet):
    MSER5YVet = []
    meiok = (len(vet)/5)/2
    meiok = int(math.ceil(meiok))
    
    for i in range(0, meiok):
        MSER5YVet.append(MSER5Y(vet, i))
    
    minimo = min(MSER5YVet)
    indexTransiente = minimo*5
    indexTransiente = int(math.ceil(indexTransiente))
    qtdMinimo = MSER5YVet.count(minimo)
    
    if(indexTransiente > len(vet)/2 or qtdMinimo > 1):
        print("Pontos de Truncagem ainda não encontrados")
        return 0
    else: 
        #print("Transiente:", indexTransiente)
        return indexTransiente

### Fila MM1

In [60]:
def process_queue(tx_service, tx_entry, removed_transient):
    # Wait = Tempo de espera total
    # tx_service = Lista com taxas de serviço
    # tx_entry = Lista com taxas de entrada
    clock = 0
    wait = 0
    wait_list = []
    bff = 0
    indexTransiente = 0
    
    for i in range(0,len(tx_service)):
        wait = clock - tx_entry[i]
        wait_list.append(wait)
        clock += tx_service[i]
            
    if(removed_transient == False):
        indexTransiente = MSER5YLoop(wait_list)
        if(indexTransiente != 0): removed_transient = True
    
    return indexTransiente, np.array(wait_list)*(-1), removed_transient

### Teste de von Neuman

In [61]:
#Vetor de tamanho B

def minorOrEqual(vet):
    R = np.zeros(len(vet))
    for i in range(0,len(vet)):
        for j in range(0, len(vet)):
            if(j != i):
                if(vet[i] >= vet[j]): R[i]+=1
    return R

def meanVon(vet):
    bff = 0
    for i in range(0, len(vet)):
        bff += vet[i]
        
    return bff/len(vet)
        
def RVN(vet):
    mean = meanVon(vet)
    sum1 = 0
    sum2 = 0
    
    for i in range(0, len(vet)-1):
        Rj = vet[i]
        Rj1 = vet[i+1]
        sum1 += (Rj - Rj1)**2
    
    for i in range(0, len(vet)):
        Rj = vet[i]
        sum2 += (Rj - mean)**2
    
    return sum1/sum2
        

def vonTest(vet):
    vetMinEq = minorOrEqual(vet)
    RNValue = RVN(vetMinEq)
    
    Beta = 1.44
    
    if(RNValue > Beta): 
        if(DEBUG == 1): print("Passou Von Test")
        return True
    else: return False


### Batch Means

In [62]:
def BatchMeans(M, B, vet):
    vetY = np.zeros(B)
    cont = 1
    j = 0
    for i in range(0, len(vet)):
        vetY[j] += vet[i]
        if(cont == M): 
            j+=1
            cont = 0
        
        cont+=1
    
    vetY = vetY/M
    GlobalMean= np.mean(vetY)
    if(DEBUG == 1): print("Global Mean: ", GlobalMean) 
    return vetY, GlobalMean

def OverlapBatchMeans(M, B, vet, settedBatch):
    N = M*B
    Blinha = N - M + 1
    vetY = np.zeros(Blinha)
    
    settedBatch = settedBatch*len(vet)
    
    for i in range(0, len(vet)):
        vetY[j] += vet[i]
        if(cont == M): 
            i = j + settedBatch
            j+=1
            cont = 0
        
        cont+=1
    
    vetY = vetY/M
    GlobalMean= np.mean(vetY)
    if(DEBUG == 1): print("Global Mean: ", GlobalMean) 
    
    return vetY, GlobalMean

### Parada

In [63]:
def Confidence_Interval_BatchMeans(vet ,RelativePrecision=0.05):
    H =  stats.t.ppf((1-RelativePrecision)/2, df=len(vet)-1)*-1*np.std(vet)
    
    Xbarra = np.mean(vet)
    
    PrecisionPass = H/Xbarra
    print('%.5f'%PrecisionPass)
    if(PrecisionPass > RelativePrecision): 
        print("Média: ", Xbarra)
        print("H:", H)
        print("n: ", len(vet))
        print("\n")
        return True
    return False

def Confidence_Interval_OverlapBatchMeans(vet ,RelativePrecision=0.05):
    df = (1.5)*(len(vet)-1)
    
    H =  stats.t.ppf((1-RelativePrecision)/2, df=df)-1*np.std(vet)
    
    Xbarra = np.mean(vet)
    
    PrecisionPass = H/Xbarra
    print('%.5f'%PrecisionPass)
    if(PrecisionPass > RelativePrecision): 
        print("Média: ", Xbarra)
        print("H:", H)
        print("n: ", len(vet))
        print("\n")
        return True
    return False

### Execução

In [64]:
def start_cicle(H, U, M=100, B=20, method = 1, settedBatch = 0.1):
    #U(μ) = taxa de entrada
    #H(λ) = taxa de serviço
    
    U = 1/U
    H = 1/H
    N = 0
    bffM = 0
    
    mean_wait = 0
    
    vonPass = False
    vonM = 0 
    vonMean = 0
    
    clock = 0
    tx_service = []
    tx_entry = []
    wait_list = []
    
    clients_list = []
    
    is_run = True #Flag para quebrar a geração infinita
    removed_transient = False
    first_remove = False
    
    interval_ok = False
    relative_presicion = False
    
    meanWait = 0
    
    GlobalMean = 0
    
    inicio = timeit.default_timer()
    
    while(is_run):
        bffM += M
        clients_list.append(bffM)
        N += M*B
        random_arr = G5(M*B)
        
    #--- Gerando vetor Taxa de serviço
        tx_service = tx_service + random_list(random_arr,U)
        
        
    #--- Gerando vetor Taxa de entrada
        bff = random_list(random_arr,H) 
        if(removed_transient): 
            tx_entry[0] = 0
            tx_entry = add_interval(tx_entry, bff)
        else:
            bff[0] = 0
            bff = sum_intervals(bff)
            tx_entry = bff
        
    #--- Processo sem remover o Transiente
        if(removed_transient == False):
            indexTransiente, wait_list, removed_transient = process_queue(tx_service, tx_entry, removed_transient)
            tx_service = tx_service[indexTransiente:]
            tx_entry = tx_entry[indexTransiente:]
            wait_list = wait_list[indexTransiente:]

    #--- Remove o Transiente e Coloca os valores restantes na lista pra seu tamanho ser igual a M*B; Ou seja, se o for removido 3 de transiente, são colocados mais 3 elementos ao final da lista.
        if(removed_transient == True and first_remove == False):    
            tx_entry[0] = 0
            tx_service, tx_entry = GenarateNValues(H, U, N, tx_service, tx_entry)
            first_remove = True
            
            
        indexTransiente, wait_list, removed_transient = process_queue(tx_service, tx_entry, removed_transient)
        
        if(method == 1): vetBatchMeans, GlobalMean = BatchMeans(bffM, B, wait_list)
        elif(method == 2): vetBatchMeans, GlobalMean = OverlapBatchMeans(bffM, B, wait_list, settedBatch)
        
        vetBatchMeans = list(vetBatchMeans)
        
        if(interval_ok == False):
            interval_ok = verify_interval(wait_list)
            if(interval_ok == True):
                if(DEBUG == 1): print("Intervalo Encontrado")
                meanWait = np.mean(vetBatchMeans)
        
        if(vonPass == False):
            vonPass = vonTest(vetBatchMeans)
            if(vonPass == True):
                vonM = bffM
                vonMean = GlobalMean
                
        if(method == 1): relative_presicion = Confidence_Interval_BatchMeans(wait_list) #BatchMeans
        elif(method == 2): relative_presicion = Confidence_Interval_OverlapBatchMeans(wait_list)
        
        
        if(relative_presicion == True):
            if(DEBUG == 1):print("Quebrado por precisão relativa")
            is_run = False
        
        if(DEBUG == 1): print("Tempo de espera por cliente: ",np.mean(wait_list)/np.mean(clients_list))
        
    fim = timeit.default_timer()
    
    return wait_list,(fim-inicio), meanWait, vonM, vonMean, clients_list

### Exercicio

In [66]:
start_cicle(7, 10)

0.03568
0.4376727349816599
0.03603
0.5821461661789433
0.03611
0.6543997094055232
0.03614
0.6977552008670606
0.03616
0.7266599836832829
0.03617
0.7473067379128108
0.03617
0.762792043979639
0.03618
0.7748363044731083
0.03618
0.7844717929994457
0.03618
0.7923554255138067
0.03619
0.798925153270982
0.03619
0.8044841772160443
0.03619
0.8092490716940579
0.03619
0.8133786592362693
0.03619
0.8169920575816537
0.03619
0.8201803573097773
0.03619
0.8230144070117782
0.03619
0.825550140034009
0.03619
0.8278323032272108
0.03619
0.8298971203565033
0.03620
0.8317742291381055
0.03620
0.8334881129694862
0.03620
0.8350591747318792
0.03620
0.8365045528835271
0.03620
0.8378387492260266
0.03620
0.8390741171686612
0.03620
0.8402212453658268
0.03620
0.8412892619820339
0.03620
0.8422860781045289
0.03620
0.8432185840442479
0.03620
0.8440928088306885
0.03620
0.8449140507079711
0.03620
0.8456869846023833
0.03620
0.8464157511670694
0.03620
0.8471040309860579
0.03620
0.847755106745297
0.03620
0.8483719155872831
0.036

KeyboardInterrupt: 

In [ ]:
start_cicle(8, 10)

In [ ]:
#start_cicle(9, 10)

In [ ]:
#start_cicle(9.5, 10)